In [1]:
# Imports
import pandas as pd
import datetime as dt

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

In [59]:
atg_obs = pd.read_csv('../../Downloads/LFEM_data-.csv', low_memory=False, skiprows=1)
atg_obs.insert(0, 'datetime_UTC', pd.to_datetime(atg_obs['Date'] + atg_obs['Time'], format='%Y-%m-%d%H:%M'))
atg_obs = atg_obs.loc[(atg_obs['datetime_UTC']>=dt.datetime(2015,5,1))].copy() # Selecting dates for training

In [60]:
atg_obs

,datetime_UTC,Pseudo-Julian-Date,Date,Time,Temperature,Dewpoint,Relhum,Speed,Gust,Direction,Pressure,Solar,SumRain,Rain
8202260,2015-05-01 00:00:00,2.457144e+06,2015-05-01,00:00,60.700000,36.400000,40.3,5.200000,6.300000,207,1018.4,367.5,585.42,0.00
8202261,2015-05-01 00:01:00,2.457144e+06,2015-05-01,00:01,60.700000,36.100000,39.7,4.300000,5.000000,217,1018.2,357.1,585.42,0.00
8202262,2015-05-01 00:02:00,2.457144e+06,2015-05-01,00:02,60.600000,36.500000,40.4,4.200000,4.900000,218,1018.3,345.8,585.42,0.00
8202263,2015-05-01 00:03:00,2.457144e+06,2015-05-01,00:03,60.600000,35.900000,39.6,5.500000,5.900000,216,1018.3,347.5,585.42,0.00
8202264,2015-05-01 00:04:00,2.457144e+06,2015-05-01,00:04,60.500000,36.000000,40.0,6.000000,6.600000,218,1018.2,362.9,585.42,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12909355,2024-04-30 19:28:00,2.460432e+06,2024-04-30,19:28,50.300000,37.100000,60.3,6.500000,7.800000,281,1015.7,269.7,967.67,0.00
12909356,2024-04-30 19:29:00,2.460432e+06,2024-04-30,19:29,50.100000,37.100000,60.8,5.000000,7.900000,358,1015.7,280.3,967.67,0.00
12909357,2024-04-30 19:30:00,2.460432e+06,2024-04-30,19:30,50.000000,37.300000,61.3,5.500000,7.400000,318,1015.7,295.8,967.67,0.00
12909358,2024-04-30 19:31:00,2.460432e+06,2024-04-30,19:31,50.000000,37.100000,61.1,8.900000,10.700000,292,1015.6,315.4,967.67,0.00


In [62]:
no_erros_obs = atg_obs.loc[(atg_obs['Rain']!='M')&(atg_obs['Temperature']!='M')&(atg_obs['Pressure']!='M')&(atg_obs['Relhum']!='M')&(atg_obs['Relhum']!='0.00.')].copy()
no_erros_obs['Rain'] = no_erros_obs['Rain'].astype(float)
no_erros_obs['Temperature'] = no_erros_obs['Temperature'].astype(float)
no_erros_obs['Pressure'] = no_erros_obs['Pressure'].astype(float)
no_erros_obs['Relhum'] = no_erros_obs['Relhum'].astype(float)

In [63]:
first_obs_each_period = no_erros_obs.resample(on='datetime_UTC', rule='1h')
first_obs_each_period

In [64]:
summed_rain_each_period = first_obs_each_period['Rain'].sum()

lightrainfall_times = summed_rain_each_period[(summed_rain_each_period<=0.10)&(summed_rain_each_period>0.0)]
moderaterainfall_times = summed_rain_each_period[(summed_rain_each_period<0.30)&(summed_rain_each_period>0.10)]
heavyrainfall_times = summed_rain_each_period[(summed_rain_each_period>0.30)]

summed_rain_each_period.loc[lightrainfall_times.index] = 1
summed_rain_each_period.loc[moderaterainfall_times.index] = 2
summed_rain_each_period.loc[heavyrainfall_times.index] = 3

In [65]:
# Extract Date, Time, Temperature, SumRain into data array
atg_obs_necessary = pd.DataFrame()
atg_obs_necessary['date'] = first_obs_each_period['Date'].first()
atg_obs_necessary['time'] = first_obs_each_period['Time'].first()
atg_obs_necessary['temperature'] = first_obs_each_period['Temperature'].first()
atg_obs_necessary['pressure'] = first_obs_each_period['Pressure'].first()
atg_obs_necessary['relhum'] = first_obs_each_period['Relhum'].first()
atg_obs_necessary['rainfall_type'] = summed_rain_each_period

data_arr = atg_obs_necessary.values
atg_obs_necessary

,date,time,temperature,pressure,relhum,rainfall_type
datetime_UTC,,,,,,
2015-05-01 00:00:00,2015-05-01,00:00,60.7,1018.4,40.3,0.0
2015-05-01 01:00:00,2015-05-01,01:00,60.8,1017.6,35.5,0.0
2015-05-01 02:00:00,2015-05-01,02:00,60.1,1017.0,35.7,0.0
2015-05-01 03:00:00,2015-05-01,03:00,59.7,1016.4,39.4,0.0
2015-05-01 04:00:00,2015-05-01,04:00,58.7,1016.4,45.6,0.0
...,...,...,...,...,...,...
2024-04-30 15:00:00,2024-04-30,15:00,46.2,1014.8,71.0,0.0
2024-04-30 16:00:00,2024-04-30,16:00,45.0,1015.1,71.9,0.0
2024-04-30 17:00:00,2024-04-30,17:00,44.6,1015.5,77.5,0.0


In [66]:
atg_obs_necessary.loc[atg_obs_necessary['rainfall_type']==3]

,date,time,temperature,pressure,relhum,rainfall_type
datetime_UTC,,,,,,
2015-08-14 20:00:00,2015-08-14,20:00,62.5,1011.6,75.6,3.0
2015-08-14 21:00:00,2015-08-14,21:00,59.4,1012.1,88.1,3.0
2015-08-30 04:00:00,2015-08-30,04:00,58.7,1004.6,85.7,3.0
2015-09-06 00:00:00,2015-09-06,00:00,58.6,1015.8,75.9,3.0
2015-10-10 17:00:00,2015-10-10,17:00,62.4,1002.0,87.0,3.0
2016-06-24 03:00:00,2016-06-24,03:00,57.1,1013.3,81.6,3.0
2016-09-02 16:00:00,2016-09-02,16:00,58.8,1009.8,76.4,3.0
2017-11-22 05:00:00,2017-11-22,05:00,56.4,1008.7,90.1,3.0
2019-08-10 14:00:00,2019-08-10,14:00,59.7,1010.8,86.8,3.0


In [67]:
date_objs = pd.to_datetime(atg_obs_necessary['date'], format="%Y-%m-%d")
time_objs = pd.to_datetime(atg_obs_necessary['time'], format='%H:%M')

days_of_year = pd.DatetimeIndex(date_objs).dayofyear
months = pd.DatetimeIndex(date_objs).month
times_in_hours_from_zero = (pd.DatetimeIndex(time_objs).hour + (pd.DatetimeIndex(time_objs).minute/60))

processed_df = pd.DataFrame()
processed_df['months_from_july'] = (np.abs(months - 7))

vals = atg_obs_necessary['temperature']
med_val = vals.median()
dif_vals = np.abs(vals-med_val)
max_normval = dif_vals.max()
norm_vals = 1-(dif_vals/max_normval)
processed_df['feature_1'] = norm_vals.values

vals = atg_obs_necessary['relhum']
med_val = vals.median()
dif_vals = np.abs(vals-med_val)
max_normval = dif_vals.max()
norm_vals = 1-(dif_vals/max_normval)
processed_df['feature_2'] = norm_vals.values

vals = atg_obs_necessary['pressure']
med_val = vals.median()
dif_vals = np.abs(vals-med_val)
max_normval = dif_vals.max()
norm_vals = 1-(dif_vals/max_normval)
processed_df['feature_3'] = norm_vals.values

processed_df['class'] = atg_obs_necessary['rainfall_type'].values
processed_df

,months_from_july,feature_1,feature_2,feature_3,class
0,2.0,0.898833,0.495017,0.871212,0.0
1,2.0,0.897536,0.415282,0.891414,0.0
2,2.0,0.906615,0.418605,0.906566,0.0
3,2.0,0.911803,0.480066,0.921717,0.0
4,2.0,0.924773,0.583056,0.921717,0.0
...,...,...,...,...,...
78903,3.0,0.913100,0.995017,0.962121,0.0
78904,3.0,0.897536,0.980066,0.954545,0.0
78905,3.0,0.892348,0.887043,0.944444,0.0
78906,3.0,0.924773,0.976744,0.939394,0.0


In [70]:
processed_df.loc[processed_df['class']==3]

,months_from_july,feature_1,feature_2,feature_3,class
2540,1.0,0.875486,0.918605,0.957071,3.0
2541,1.0,0.915694,0.710963,0.969697,3.0
2908,1.0,0.924773,0.750831,0.780303,3.0
3072,2.0,0.926070,0.913621,0.936869,3.0
3905,3.0,0.876783,0.729236,0.714646,3.0
10083,1.0,0.945525,0.818937,1.000000,3.0
11776,2.0,0.923476,0.905316,0.911616,3.0
22469,4.0,0.954604,0.677741,0.883838,3.0
37502,1.0,0.911803,0.732558,0.936869,3.0
38187,2.0,0.837873,0.973422,0.924242,3.0


In [71]:
processed_data = processed_df.values.astype(float)

In [72]:
processed_data

array([[2.        , 0.89883268, 0.49501661, 0.87121212, 0.        ],
       [2.        , 0.89753567, 0.41528239, 0.89141414, 0.        ],
       [2.        , 0.90661479, 0.41860465, 0.90656566, 0.        ],
       ...,
       [3.        , 0.8923476 , 0.88704319, 0.94444444, 0.        ],
       [3.        , 0.92477302, 0.97674419, 0.93939394, 0.        ],
       [3.        , 0.95330739, 0.81395349, 0.94191919, 0.        ]])

In [73]:
print(f'Data Shape: {data_arr.shape}')
print(f'Data Entry: [Date, Time, Temperature, Rain]')

# Pre-Processed
print(data_arr[0])

# Post-Processed
print(processed_data[0])

Data Shape: (78908, 6)
Data Entry: [Date, Time, Temperature, Rain]
['2015-05-01' '00:00' 60.7 1018.4 40.3 0.0]
[2.         0.89883268 0.49501661 0.87121212 0.        ]


In [74]:
x_data = processed_data[:, :5]
y_data = processed_data[:, 5:]
print(x_data[0], y_data[0])

[2.         0.89883268 0.49501661 0.87121212 0.        ] []


In [75]:
# Simple Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

In [76]:
# Train the Model:
model.fit(x_data, y_data, epochs=5)

Epoch 1/5
   1/2466 [..............................] - ETA: 13:08 - loss: 0.0000e+00 - mae: 0.0000e+00

2024-05-19 22:44:39.629950: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2466/2466 [==============================] - 36s 15ms/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 2/5
2466/2466 [==============================] - 37s 15ms/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 3/5
2466/2466 [==============================] - 37s 15ms/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 4/5
2466/2466 [==============================] - 39s 16ms/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 5/5
 747/2466 [========>.....................] - ETA: 26s - loss: 0.0000e+00 - mae: 0.0000e+00

In [ ]:
# Evaluation (Is it worth, the loss is quite high suggesting that either the model architecture is quite poor (likely) or 
# that we need to better preprocess our data such as rain (also likely))